In [19]:
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AdamW, AutoTokenizer, AutoConfig, AutoModelForSeq2SeqLM, AutoModelForCausalLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
import numpy as np
import evaluate
import torch
from torch import nn
from torch.utils.data import DataLoader

In [3]:
notebook_login()

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [194]:
# tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-40b", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-40b", trust_remote_code=True, num_labels=5)
# config = AutoConfig.from_pretrained("tiiuae/falcon-40b", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", num_labels=5)
config = AutoConfig.from_pretrained("google/flan-t5-base")

In [56]:
# dataset = load_dataset("squad")
dataset = load_dataset("medmcqa")

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

In [57]:
dataset.shape

{'train': (182822, 11), 'test': (6150, 11), 'validation': (4183, 11)}

In [211]:
formatted_dataset = dataset.filter(lambda example: example["exp"]!=None and "Ans." not in example["exp"])
# formatted_dataset = formatted_dataset.map(
#     lambda example: {
#         "context": example["exp"],
#         "question": example["question"],
#         "opa": example["opa"],
#         "opb": example["opb"],
#         "opc": example["opc"],
#         "opd": example["opd"],
#         "answer_key": example["cop"]
#     }
# )
def map_function(example):
    correct_answer_key = "opa"
    if example["cop"] == 1:
        correct_answer_key = "opb"
    elif example["cop"] == 2:
        correct_answer_key = "opc"
    else:
        correct_answer_key = "opd"
    option_a = example['opa']
    example['opa'] = example[correct_answer_key]
    example[correct_answer_key] = option_a
    return {
        "exp": example["exp"],
        "question": example["question"],
        "opa": example["opa"],
        "opb": example["opb"],
        "opc": example["opc"],
        "opd": example["opd"]
    }
formatted_dataset = formatted_dataset.map(map_function)

Map:   0%|          | 0/118404 [00:00<?, ? examples/s]

Map:   0%|          | 0/6150 [00:00<?, ? examples/s]

Map:   0%|          | 0/1931 [00:00<?, ? examples/s]

In [231]:
def tokenize_function(examples):
#     text = f"{examples['context']} Question: {examples['question']} Choices: {' '.join((examples['opa'], examples['opb'], examples['opc'], examples['opd']))} Answer: {examples['answer_key']}"
#     text = f"{examples['question']} {' '.join((examples['opa'], examples['opb'], examples['opc'], examples['opd']))} Answer: {examples['answer_key']}"
#     tokens = tokenizer(text, truncation=True, padding="max_length")
#     labels = tokens['input_ids']
    inputs = tokenizer(examples['exp'], padding="max_length", truncation=True, max_length=256)
    question_input = tokenizer(examples['question'], padding="max_length", truncation=True, max_length=256)
    opa_input = tokenizer(examples['opa'], padding="max_length", truncation=True, max_length=256)
    opb_input = tokenizer(examples['opb'], padding="max_length", truncation=True, max_length=256)
    opc_input = tokenizer(examples['opc'], padding="max_length", truncation=True, max_length=256)
    opd_input = tokenizer(examples['opd'], padding="max_length", truncation=True, max_length=256)
    labels_matrix = np.zeros((len(examples['exp']), 5, 256))
    labels_matrix[:, 0] = question_input.input_ids
    labels_matrix[:, 1] = opa_input.input_ids
    labels_matrix[:, 2] = opb_input.input_ids
    labels_matrix[:, 3] = opc_input.input_ids
    labels_matrix[:, 4] = opd_input.input_ids
    inputs['labels'] = labels_matrix.tolist()
#     inputs['labels'] = tokenizer(f"#question#> {examples['question']} #opa#> {examples['opa']} #opb#> {examples['opb']} #opc#> {examples['opc']} #opd#> {examples['opd']}").input_ids
    return inputs
#     labels = {'question': question_input, "opa": opa_input, "opb": opb_input, "opc": opc_input, "opd": opd_input, "answer_key": answer_key_input}
    
#     inputs = tokens

#     return {
#         'input_ids': inputs['input_ids'],
#         'attention_mask': inputs['attention_mask'],
#         'labels': labels,
#     }
# def tokenize_function(examples):
#     return {"input_ids": tokenizer(examples['context']).input_ids, 
#             "labels": tokenizer("Question: {} Answer: {}".format(examples['question'], examples['answers']['text'][0])).input_ids}

In [232]:
tokenized_datasets = formatted_dataset.map(tokenize_function, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/118404 [00:00<?, ? examples/s]

Map:   0%|          | 0/6150 [00:00<?, ? examples/s]

Map:   0%|          | 0/1931 [00:00<?, ? examples/s]

In [247]:
print(tokenized_datasets['validation'][0])

{'input_ids': [389, 7, 18, 9, 5, 37, 30, 509, 1225, 1666, 13, 8, 5798, 3140, 8, 212, 12851, 725, 19, 705, 145, 24, 13, 5798, 7084, 34, 2846, 21220, 27, 301, 102, 683, 536, 7769, 5, 117, 152, 2444, 1902, 87, 15, 3, 102, 4122, 25342, 834, 61, 517, 40, 32, 935, 4885, 30, 509, 1225, 1666, 41, 1259, 15, 12, 18309, 3619, 738, 61, 19, 1146, 145, 24, 13, 15775, 342, 30, 509, 1225, 1666, 16, 10715, 348, 31, 7, 13445, 121, 134, 77, 565, 17320, 19, 14019, 3, 23161, 11, 8, 5798, 16, 8, 10715, 348, 31, 7, 13445, 19, 19, 32, 17, 4554, 28, 18309, 6, 8, 6145, 13, 17320, 16, 8, 15775, 342, 19, 8, 337, 38, 16, 8, 212, 12851, 725, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [242]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(50))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(50))

In [134]:
text = "Create a multiple choice question from the following text: My name is bob. I have five cats."
tokenized_text = tokenizer(text, return_tensors='pt').input_ids
outputs = model.generate(tokenized_text)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

D:\miniconda3\envs\read-for-speed\Lib\site-packages\transformers\generation\utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Question: What is bob's name?Options:A BobB BobC


In [243]:
batch_size = 4
# args = Seq2SeqTrainingArguments(
args = TrainingArguments(
    "read-for-speed-test-model",
    evaluation_strategy = "epoch",
    learning_rate=2e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
#     predict_with_generate=True,
#     push_to_hub=True,
    push_to_hub=False
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [244]:
# trainer = Seq2SeqTrainer(
trainer = Trainer(
    model,
    args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.092706
2,No log,0.940038


TrainOutput(global_step=26, training_loss=1.0821060767540565, metrics={'train_runtime': 972.6663, 'train_samples_per_second': 0.103, 'train_steps_per_second': 0.027, 'total_flos': 34237867622400.0, 'train_loss': 1.0821060767540565, 'epoch': 2.0})

In [246]:
text = "What is up? the sky"
tokenized_text = tokenizer(text, return_tensors='pt').input_ids
outputs = model.generate(tokenized_text)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tensor([[   0, 1713,    2,  994,  102, 4663, 3155,  363,   19,   95,   58,    8,
         5796, 1713,    2, 7771, 1575, 4663, 3155,  363]])
#exp#> What is up? the sky #question#> What


In [13]:
trainer.push_to_hub("End of training")

Upload file pytorch_model.bin:   0%|          | 1.00/945M [00:00<?, ?B/s]

Upload file runs/Oct17_13-39-37_DESKTOP-Q7BHUCL/events.out.tfevents.1697564386.DESKTOP-Q7BHUCL.17232.0:   0%| …

Upload file spiece.model:   0%|          | 1.00/773k [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/4.06k [00:00<?, ?B/s]

To https://huggingface.co/AJCheu/read-for-speed-test-model
   02df199..f1b38f6  main -> main

To https://huggingface.co/AJCheu/read-for-speed-test-model
   f1b38f6..7952045  main -> main



'https://huggingface.co/AJCheu/read-for-speed-test-model/commit/f1b38f65b98e61fb4d7bb9de37b296c4118abd12'

In [3]:
class CustomT5ForTextGeneration(torch.nn.Module):
    def __init__(self, num_labels, t5_model_name="google/flan-t5-base"):
        super(CustomT5ForTextGeneration, self).__init__()
        
        # Load the pre-trained T5 model
        self.t5 = T5ForConditionalGeneration.from_pretrained(t5_model_name)
        for param in self.t5.parameters():
             param.requires_grad = False
        
        # Define a new model head for text generation
        self.text_generation_head = torch.nn.Linear(self.t5.config.d_model, num_labels)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, **kwargs):
        # Use the T5 model to get hidden states
        outputs = self.t5(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, **kwargs)
        
        # Pass the hidden states through the text generation head
        text_outputs = self.text_generation_head(outputs.last_hidden_state)
        
        return text_outputs

num_labels = 5  # Number of text generation outputs
model = CustomT5ForTextGeneration(num_labels)

In [4]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [5]:
dataset = load_dataset("medmcqa")

In [6]:
formatted_dataset = dataset.filter(lambda example: example["exp"]!=None and "Ans." not in example["exp"])
def map_function(example):
    correct_answer_key = "opa"
    if example["cop"] == 1:
        correct_answer_key = "opb"
    elif example["cop"] == 2:
        correct_answer_key = "opc"
    else:
        correct_answer_key = "opd"
    option_a = example['opa']
    example['opa'] = example[correct_answer_key]
    example[correct_answer_key] = option_a
    return {
        "exp": example["exp"],
        "question": example["question"],
        "opa": example["opa"],
        "opb": example["opb"],
        "opc": example["opc"],
        "opd": example["opd"]
    }
formatted_dataset = formatted_dataset.map(map_function)

In [9]:
def tokenize_function(examples):
    inputs = tokenizer(examples['exp'], padding="max_length", truncation=True, max_length=256)
    question_input = tokenizer(examples['question'], padding="max_length", truncation=True, max_length=256)
    opa_input = tokenizer(examples['opa'], padding="max_length", truncation=True, max_length=256)
    opb_input = tokenizer(examples['opb'], padding="max_length", truncation=True, max_length=256)
    opc_input = tokenizer(examples['opc'], padding="max_length", truncation=True, max_length=256)
    opd_input = tokenizer(examples['opd'], padding="max_length", truncation=True, max_length=256)
    labels_matrix = np.zeros((len(examples['exp']), 5, 256))
    labels_matrix[:, 0] = question_input.input_ids
    labels_matrix[:, 1] = opa_input.input_ids
    labels_matrix[:, 2] = opb_input.input_ids
    labels_matrix[:, 3] = opc_input.input_ids
    labels_matrix[:, 4] = opd_input.input_ids
    inputs['labels'] = labels_matrix.tolist()
    return inputs

In [10]:
tokenized_datasets = formatted_dataset.map(tokenize_function, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/118404 [00:00<?, ? examples/s]

Map:   0%|          | 0/6150 [00:00<?, ? examples/s]

Map:   0%|          | 0/1931 [00:00<?, ? examples/s]

In [11]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(50))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(50))

In [14]:
train_dataloader = DataLoader(small_train_dataset, batch_size=2, shuffle=True)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=2)

In [21]:
t5_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

In [22]:
custom_head = torch.nn.Linear(t5_model.config.d_model, 5)

In [20]:
optimizer = AdamW(custom_head.parameters(), lr=1e-4)

D:\miniconda3\envs\read-for-speed\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
num_epochs = 5
eval_interval = 1

for epoch in range(num_epochs):
    total_loss = 0.0

    # Training phase
    t5_model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_batch, attention_mask, target_batch = batch.values()
        # Forward pass through the T5 model
        t5_outputs = t5_model(input_ids=input_batch, attention_mask=attention_mask)

        # Pass T5 outputs through the custom head
        custom_head_outputs = custom_head(t5_outputs.last_hidden_state)

        # Calculate loss and perform backpropagation
        loss = torch.nn.functional.cross_entropy(custom_head_outputs, target_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Train Loss: {average_loss:.4f}")

    # Evaluation phase
    if (epoch + 1) % eval_interval == 0:
        t5_model.eval()
        eval_loss = 0.0

        with torch.no_grad():
            for batch in eval_dataloader:
                input_batch, attention_mask, target_batch = batch.values()

                # Forward pass during evaluation
                t5_outputs = t5_model(input_ids=input_batch, attention_mask=attention_mask)
                custom_head_outputs = custom_head(t5_outputs.last_hidden_state)

                # Calculate evaluation loss
                loss = torch.nn.functional.cross_entropy(custom_head_outputs, target_batch)
                eval_loss += loss.item()

        average_eval_loss = eval_loss / len(eval_dataloader)
        print(f"Epoch {epoch + 1}, Evaluation Loss: {average_eval_loss:.4f}")

AttributeError: 'list' object has no attribute 'size'

In [32]:
testingmodel = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", num_labels=5)

AttributeError: 'CustomT5ForTextGeneration' object has no attribute 'generate'

In [33]:
output = testingmodel.generate(tokenizer("translate English to German: How old are you?", return_tensors="pt").input_ids)

D:\miniconda3\envs\read-for-speed\Lib\site-packages\transformers\generation\utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [35]:
print(tokenizer.decode(output[0]))

<pad> Wie old sind Sie?</s>
